# Импорты нужных библиотек

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import statsmodels.api as sm
from IPython.display import display, Math, Markdown
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
ip = get_ipython() 
ibe = ip.configurables[-1] 
ibe.figure_formats = { 'pdf', 'png'}

# Подготовка данных 

In [42]:
# загружаем файлы xlsx как таблицы pd.Dataframe
rev_2017m = pd.read_excel('2017.xlsx')
rev_2018m = pd.read_excel('2018.xlsx')
gdp = pd.read_excel('gdp.xls')

# убираем пробелы в конце строк в названиях стран
rev_2017m['COUNTRY'] = rev_2017m['COUNTRY'].apply(lambda x: x.rstrip())
rev_2018m['COUNTRY'] = rev_2018m['COUNTRY'].apply(lambda x: x.rstrip())

# меняем название столбца
rev_2017m.rename(columns={'TOTAL REVENUES ($M)':'TOTAL REVENUES'},inplace=True)

# добавляем столбец "год"
rev_2017m['YEAR'] = [2017]*(rev_2017m.shape[0])
rev_2018m['YEAR'] = [2018]*(rev_2018m.shape[0])

# делаем значения объема продаж числами 
# (например,строка "$2,303M" для таблицы rev_2018m становится числом 2303)
rev_2017m['TOTAL REVENUES'] = rev_2017m['TOTAL REVENUES'].apply(lambda x: int(x.replace(',','_')[1:]))
rev_2018m['TOTAL REVENUES'] = rev_2018m['TOTAL REVENUES'].apply(lambda x: int(x.replace(',','_')[1:-1]))

# создаем таблицы с уровнем ВВП для 2017 и 2018 года, переводим доллары в миллионы долларов,
# переводим буквы в названиях стран в верхний регистр ("Russian Federation" => "RUSSIAN FEDERATION")
gdp_2017m = gdp.loc[:][['COUNTRY',2017]]
gdp_2017m.loc[:][2017] /= 10**6
gdp_2017m.loc[:]['COUNTRY'] = gdp_2017m.loc[:]['COUNTRY'].apply(lambda x: x.upper()) 

gdp_2018m = gdp.loc[:][['COUNTRY',2018]]
gdp_2018m.loc[:][2018] /= 10**6
gdp_2018m.loc[:]['COUNTRY'] = gdp_2018m.loc[:]['COUNTRY'].apply(lambda x: x.upper())

# меняем названия столбцов
gdp_2017m.rename(columns={2017:'GDP'},inplace=True)
gdp_2018m.rename(columns={2018:'GDP'},inplace=True)

# соединяем таблицы продаж с таблицами ввп соответсвенно (аналогично функции join в SQL)
data1 = rev_2017m.merge(gdp_2017m,on='COUNTRY')
data2 = rev_2018m.merge(gdp_2018m,on='COUNTRY')

# соединяем получившиеся таблицы (аналогично функции union в SQL) и удаляем пустые значения
data = data1.append(data2).dropna()

# сохраняем в файл xlsx
data.to_excel('data.xlsx',index=False)

# Оценка модели

In [29]:
from sklearn.model_selection import train_test_split as tsplit

# логарифмируем значения Y и R
x = np.log(data['TOTAL REVENUES'].values)
y = np.log(data['GDP'].values)

# добавляем столбец с единицами для свободного члена
x = np.concatenate((np.array([1]*x.shape[0]).reshape(-1,1),x.reshape(-1,1)),axis=1)

# формируем тренировочную и контрольную выборки (объем контрольной выборки 10%)
x_train, x_test, y_train, y_test = tsplit(x, y, test_size=0.1, random_state=1)

# оцениванием модель с помощью МНК
gls = sm.OLS(y_train, x_train)
result = gls.fit()

# удобное отображение параметров оценки
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     394.4
Date:                Wed, 18 Dec 2019   Prob (F-statistic):           3.36e-24
Time:                        21:11:11   Log-Likelihood:                -22.563
No. Observations:                  48   AIC:                             49.13
Df Residuals:                      46   BIC:                             52.87
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.0565      0.301     26.784      0.000       7.451       8.662
x1             0.8233      0.041     19.860      0.000       0.740       0.907
==============================================================================
Omnibus:                        4.825   Durbin-Watson:                   1.589
Prob(Omnibus):                  0.090   Jarque-Bera (JB):                5.138
Skew:                           0.233   Prob(JB):                       0.0766
Kurtosis:                       4.534   Cond. No.                         38.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Проверка предпосылок
## 1)F-test

In [26]:
# объем обучающей выборки
n = len(x_train)

# вычисление k
# если первый столбец x_train содержит только единицы, то имеется свободный член
if np.array_equal( x_train[:,0].astype(int), np.array([1]*x_train.shape[0]) ):
    k = x_train.shape[1]-1
    a0 = 1 #наличие свободного члена
else:
    k = x_train.shape[1]
    a0 = 0

# удобный вывод проверки
Markdown('$F$ = '+str(result.fvalue))
Markdown('$F_{крит}$ = '+str(st.f.isf(0.05,k,n-k-a0)))
if result.fvalue >= st.f.isf(0.05,k,n-k-a0):
    Markdown('На уровне значимости 5% гипотеза о качественности модели не отвергается')
else:
    Markdown('На уровне значимости 5% гипотеза о качественности модели отвергается')
Markdown(r'$E(\tilde u_{c,t})$ = '+str(np.array([y - y_ for y,y_ in zip(y_train,y_pred_train)]).mean()))

$F$ = 394.40536984048885

$F_{крит}$ = 4.051748692149202

На уровне значимости 5% гипотеза о качественности модели не отвергается

$E(\tilde u_{c,t})$ = -7.919590908992783e-15

## 3)Тест Бройша — Годфри

In [36]:
Markdown('$BG$ = '+str(sm.stats.diagnostic.acorr_breusch_godfrey(result,1)[0]))
Markdown('$\chi^2_{0.05}$ ='+str(st.chi2.isf(0.05,1)))
if sm.stats.diagnostic.acorr_breusch_godfrey(result,1)[0] < st.chi2.isf(0.05,1):
    Markdown('На уровне значимости 5% гипотеза об отсутствии автокорреляции случайных остатков не отвергается')
else:
    Markdown('На уровне значимости 5% гипотеза об отсутствии автокорреляции случайных остатков отвергается')

$BG$ = 1.9993268916726556

$\chi^2_{0.05}$ =3.8414588206941285

На уровне значимости 5% гипотеза об отсутствии автокорреляции случайных остатков не отвергается

# Проверка адекватности

In [41]:
# прогнозы Y на основе обучающей и контрольной выборки
y_pred_train = result.predict(x_train)
y_pred_test = result.predict(x_test)

# список для подсчета прогнозов, попавших в доверительный интервал
scores = np.array([0]*len(y_pred_test))

# считаем данные для построения доверительных интервалов 
sigma = ( sum((y-y_)**2 for y,y_ in zip(y_train,y_pred_train)) / (n-k-a0))**.5
t = st.t.isf(0.025,n-k-a0)

for i,y0 in enumerate(y_pred_test):
    
    q0 = 1/n + ((x_test[i].mean()-x_train.mean())**2/sum((x - x_train[:,1].mean())**2 for x in x_train[:,1]))
    s_y0 = sigma*(q0+1)**0.5
    
    # проверяем, попадает ли элемент контрольной выборки в доверительный интервал
    if y0-s_y0*t <= y_test[i] <= y0+s_y0*t:
        scores[i] = 1

Markdown('На основе контрольной выборки объема '\
         +str(len(y_pred_test))+' точность прогнозов:$'+str(scores.mean()*100)+'$%')

На основе контрольной выборки объема 6 точность прогнозов:$100.0$%